In [1]:
import sys
sys.path.append("C:/Users/user/meepc")
import numpy as np
import pandas as pd
from models import Hankel,Corrhankel,RobustPipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def calculate_fpr(y_actual, y_prediction):
    fp = 0
    tn = 0
    
    for i in range(len(y_actual)):
        if y_actual[i] == 0 and y_prediction[i] == 1:
            fp += 1
        elif y_actual[i] == 0 and y_prediction[i] == 0:
            tn += 1
    fpr= fp/(fp+tn)
    return fpr

In [3]:
train_normal = pd.read_csv('~/data/te/normal_training.csv')
train_attack = pd.read_csv('~/data/te/attack_training.csv')
train_normal = pd.concat((train_normal,train_attack),axis=0,ignore_index=True)
y_truth = train_normal.loc[:,'LABEL']

In [4]:
hankel = Hankel()
corrhankel = Corrhankel()
lag = 500
stride = 0.5

In [5]:
#attack indexes in original training data
temp = hankel.fit(np.array(y_truth) ,lag,stride)
attack_idx = np.any(temp>0,axis=0).astype(int)
attack_idx = np.where(attack_idx>0)[0]
attack_idx

array([ 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009,
       1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020,
       1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031,
       1032, 1033, 1034, 1035, 1036, 1037], dtype=int64)

In [6]:
sensors = [col for col in train_normal.columns if col not in ['faultNumber', 'simulationRun', 'sample','LABEL']]

In [7]:
scaler = StandardScaler()
X_normal = pd.DataFrame(index=train_normal.index, columns=sensors, data=scaler.fit_transform(train_normal[sensors]))
X_attack = pd.DataFrame(data=scaler.fit_transform(train_attack.loc[:,sensors]),index=train_attack.index,columns=sensors).reset_index().drop(columns=['index'])

In [8]:
corr_normal,nolag_normal = corrhankel.fit(X_normal.to_numpy(),lag,stride)

In [9]:
corr_attack,nolag_attack = corrhankel.fit(X_attack.to_numpy(),lag,stride)

In [10]:
df_test1 = pd.read_csv('~/data/te/normal_testing.csv')
df_test1 = df_test1.sample(n=15000, random_state=42)
df_test1.reset_index(drop=True, inplace=True)
df_test2 = pd.read_csv('~/data/te/attack_testing.csv')
df_test2 = df_test2.groupby('faultNumber').apply(lambda x: x.sample(n=150, random_state=42))
df_test2.reset_index(drop=True, inplace=True)
df_test=pd.concat((df_test1,df_test2),axis=0)

# Epasad with 1 cluster

In [11]:
X_test = pd.DataFrame(index=df_test.index, columns=sensors, data=scaler.fit_transform(df_test[sensors]))
corr_test,nolag_test = corrhankel.fit(X_test.to_numpy(),lag,stride)
Y_test = df_test.loc[:,'LABEL']

In [12]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [13]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
fpr = []
cluster_indexes=[]
pca_counts=0
meepc_counts=0

for i,sens in enumerate(sensors):
    print(sens)
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = RobustPipeline()
    model.fit(train_normal,train_attack,lag,stride,optimal_k=1,kscore_init='inertia',tune=False,alpha=0.15,corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,y_truth=y_truth)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_normal.columns)).T)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    fpr.append(calculate_fpr(y_actual,y_predicted))
    sensor_models.append(model)    

    #clustering
    cluster_indexes.append(model.cluster_idx)
    #pca
    pca_counts=max(pca_counts,model.total_pca_attacks)
    #meepc
    meepc_counts=max(meepc_counts,model.counts_meepc)

xmeas_1
------[CLUSTER] attack points found is 18 
------[PCA] attack points found is 39 
------[Meepc] attack points found is 36 
xmeas_2
------[CLUSTER] attack points found is 13 
------[PCA] attack points found is 39 
------[Meepc] attack points found is 3 
xmeas_3
------[CLUSTER] attack points found is 19 
------[PCA] attack points found is 39 
------[Meepc] attack points found is 18 
xmeas_4
------[CLUSTER] attack points found is 21 
------[PCA] attack points found is 39 
------[Meepc] attack points found is 38 
xmeas_5
------[CLUSTER] attack points found is 6 
------[PCA] attack points found is 39 
------[Meepc] attack points found is 2 
xmeas_6
------[CLUSTER] attack points found is 17 
------[PCA] attack points found is 39 
------[Meepc] attack points found is 0 
xmeas_7
------[CLUSTER] attack points found is 33 
------[PCA] attack points found is 39 
------[Meepc] attack points found is 27 
xmeas_8
------[CLUSTER] attack points found is 16 
------[PCA] attack points found is 3

In [14]:
cluster_inactive_idx = np.unique(np.concatenate(cluster_indexes))
commonidxs=np.intersect1d(attack_idx,cluster_inactive_idx)
percentage = len(commonidxs)/len(attack_idx)*100
print(percentage)

100.0


In [15]:
percentage = pca_counts/len(attack_idx)*100
print(percentage)

100.0


In [16]:
percentage = meepc_counts/len(attack_idx)*100
print(percentage) 

100.0


In [17]:
sensor_predicted = np.asarray(sensor_predicted)

In [ ]:
# sensor_mean = np.mean(sensor_predicted, axis=0)
# y_predicted = (sensor_mean > 0.3).astype(int)
# y_predicted 

In [18]:
y_predicted = np.any(sensor_predicted,axis=0).astype(int)
y_predicted

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1])

In [19]:
print("Accuracy ",accuracy_score(y_actual,y_predicted))
print("Precision ",precision_score(y_actual,y_predicted))
print("Recall ",recall_score(y_actual,y_predicted))
print("F1-score ",f1_score(y_actual,y_predicted))
print("False Positive Rate ",calculate_fpr(y_actual,y_predicted))

Accuracy  0.9428571428571428
Precision  0.8181818181818182
Recall  0.8181818181818182
F1-score  0.8181818181818182
False Positive Rate  0.03389830508474576


In [ ]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.8565934065934064 Median : 0.8428571428571429 Min : 0.8428571428571429 Max : 0.9142857142857143
precision - Mean: 0.37820512820512814 Median : 0.0 Min : 0.0 Max : 1.0
recall -    Mean: 0.09265734265734264 Median : 0.0 Min : 0.0 Max : 0.45454545454545453
f1 -        Mean: 0.1439173006480699 Median : 0.0 Min : 0.0 Max : 0.625


# Multiple clusters + No threshold tuning

In [ ]:
cluster_accuracy = []
cluster_precision = []
cluster_recall = []
cluster_fscore = []
cluster_fpr = []
for k in range(2,6):
    print('-------------------------------------',k,'-------------------------------------')
    
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    fpr = []
    cluster_indexes=[]
    pca_counts=0
    meepc_counts=0
    for i,sens in enumerate(sensors):
        print(sens)
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = RobustPipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k=k,tune=False,kscore_init='inertia',alpha=0.15,corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,y_truth=y_truth)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_normal.columns)).T)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        fpr.append(calculate_fpr(y_actual,y_predicted))
        sensor_models.append(model) 
        cluster_indexes.append(model.cluster_idx)
        pca_counts=max(pca_counts,model.total_pca_attacks)
        meepc_counts=max(meepc_counts,model.counts_meepc)
           
    #cluster
    cluster_inactive_idx = np.unique(np.concatenate(cluster_indexes))
    commonidxs=np.intersect1d(attack_idx,cluster_inactive_idx)
    percentage = len(commonidxs)/len(attack_idx)*100
    print("Cluster - attack points detected {} %".format(percentage))
    #PCA
    
    percentage = pca_counts/len(attack_idx)*100
    print("PCA - attack points detected {} %".format(percentage))
    #Meepc
    percentage = meepc_counts/len(attack_idx)*100
    print("Meepc - attack points detected {} %".format(percentage),"/n")
            
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    cluster_accuracy.append(accuracy_score(y_actual, y_predicted))
    cluster_precision.append(precision_score(y_actual,y_predicted))
    cluster_recall.append(recall_score(y_actual,y_predicted))
    cluster_fscore.append(f1_score(y_actual,y_predicted))
    cluster_fpr.append(calculate_fpr(y_actual,y_predicted))
    print('------Number of Clusters: ',k,'-----------') 
    print("Accuracy ",cluster_accuracy[-1])
    print("Precision ",cluster_precision[-1])
    print("Recall ",cluster_recall[-1])
    print("F1-score ",cluster_fscore[-1])
    print("False Positive Rate ",cluster_fpr[-1],"\n")

------------------------------------- 2 -------------------------------------
xmeas_1
------[CLUSTER] attack points found is 18 
------[PCA] attack points found is 10 
------[Meepc] attack points found is 7 
------[PCA] attack points found is 29 
------[Meepc] attack points found is 16 
xmeas_2
------[CLUSTER] attack points found is 13 
------[PCA] attack points found is 26 
------[Meepc] attack points found is 1 
------[PCA] attack points found is 13 
------[Meepc] attack points found is 2 
xmeas_3
------[CLUSTER] attack points found is 19 
------[PCA] attack points found is 28 
------[Meepc] attack points found is 6 
------[PCA] attack points found is 11 
------[Meepc] attack points found is 3 
xmeas_4
------[CLUSTER] attack points found is 21 
------[PCA] attack points found is 24 
------[Meepc] attack points found is 15 
------[PCA] attack points found is 15 
------[Meepc] attack points found is 13 
xmeas_5
------[CLUSTER] attack points found is 7 
------[PCA] attack points found i

In [ ]:
print("Accuracy Scores: ",cluster_accuracy)
print("Precision Scores: ",cluster_precision)
print("Recall Scores: ",cluster_recall)
print("F1 Scores: ",cluster_fscore)
print("False Positive Rates: ",cluster_fpr)

Accuracy Scores:  [0.9142857142857143, 1.0, 1.0, 0.9857142857142858, 0.9714285714285714, 0.9714285714285714]
Precision Scores:  [0.6470588235294118, 1.0, 1.0, 1.0, 1.0, 1.0]
Recall Scores:  [1.0, 1.0, 1.0, 0.9090909090909091, 0.8181818181818182, 0.8181818181818182]
F1 Scores:  [0.7857142857142858, 1.0, 1.0, 0.9523809523809523, 0.9, 0.9]
False Positive Rates:  [0.1016949152542373, 0.0, 0.0, 0.0, 0.0, 0.0]
